# TYPO CORRECTION PROJECT (TC4)

### LOADING THE DATA SET

In [94]:
import pickle
import numpy as np
from collections import defaultdict

In [95]:
original = "./data/train10.pkl"
destination = "./data/train10_.pkl"

content = ''
outsize = 0
with open(original, 'rb') as infile:
    content = infile.read()
with open(destination, 'wb') as output:
    for line in content.splitlines():
        outsize += len(line) + 1
        output.write(line + str.encode('\n'))


In [96]:
original = "./data/test10.pkl"
destination = "./data/test10_.pkl"

content = ''
outsize = 0
with open(original, 'rb') as infile:
    content = infile.read()
with open(destination, 'wb') as output:
    for line in content.splitlines():
        outsize += len(line) + 1
        output.write(line + str.encode('\n'))


In [97]:
with open("./data/train10_.pkl", "rb") as f:
    train_data = pickle.load(f)

with open("./data/test10_.pkl", "rb") as f:
    test_data = pickle.load(f)   



In [98]:
original = "./data/train20.pkl"
destination = "./data/train20_.pkl"

content = ''
outsize = 0
with open(original, 'rb') as infile:
    content = infile.read()
with open(destination, 'wb') as output:
    for line in content.splitlines():
        outsize += len(line) + 1
        output.write(line + str.encode('\n'))
        
        
original = "./data/test20.pkl"
destination = "./data/test20_.pkl"

content = ''
outsize = 0
with open(original, 'rb') as infile:
    content = infile.read()
with open(destination, 'wb') as output:
    for line in content.splitlines():
        outsize += len(line) + 1
        output.write(line + str.encode('\n'))
        
with open("./data/train20_.pkl", "rb") as f:
    train_data_20 = pickle.load(f)

with open("./data/test20_.pkl", "rb") as f:
    test_data_20 = pickle.load(f)   


In [99]:
class Dict:
    def __init__(self, chars, unk=None):
        self._unk = unk
        self._char_to_id = dict()
        self._id_to_char = list()

        if unk in chars:
            raise RuntimeError("UNK word exists in vocabulary")

        if unk is not None:
            self.unk_index = self._add_char(unk)

        for char in chars:
            self._add_char(char)

    # for internal use only!
    def _add_char(self, char):
        if char not in self._char_to_id:
            id = len(self._id_to_char)
            self._char_to_id[char] = id
            self._id_to_char.append(char)
            return id
        else:
            return self._char_to_id[char]

    def char_to_id(self, char):
        if self._unk is not None:
            return self._char_to_id.get(char, self.unk_index)
        else:
            return self._char_to_id[char]

    def id_to_char(self, id):
        return self._id_to_char[id]

    def __len__(self):
        return len(self._char_to_id)

    def has_unk(self):
        return self._unk is not None
    
    def unk(self):
        return self.unk_index

# TYPOS CORRECTION WITH 10 % ERROR RATE

In [100]:
distribution_per_char = {}
distribution_per_char_correct = {}
for w in train_data:
    for typed_c, correct_c in w:
        if typed_c in distribution_per_char : 
            if correct_c in distribution_per_char[typed_c]:
                distribution_per_char[typed_c][correct_c]+=1
            else:
                distribution_per_char[typed_c][correct_c] = 1
        else:
            distribution_per_char[typed_c]={}
            distribution_per_char[typed_c][correct_c] = 1

In [101]:
print(distribution_per_char)

{'b': {'b': 1843, 'n': 233, 'g': 44, 'h': 129, 'v': 40}, 'y': {'y': 2684, 'h': 118, 't': 354, 'u': 110}, 't': {'t': 12540, 'r': 207, 'f': 64, 'y': 76, 'g': 54}, 'h': {'h': 6047, 'n': 230, 'b': 73, 'y': 73, 'u': 90, 'g': 43, 'j': 2}, 'e': {'e': 16274, 'd': 97, 'r': 203, 'w': 64}, 'i': {'i': 9856, 'o': 302, 'u': 97, 'k': 19}, 'r': {'r': 7416, 'e': 624, 'f': 69, 't': 334}, 'o': {'o': 10720, 'p': 163, 'i': 273, 'l': 186}, 'w': {'w': 1999, 'e': 581, 's': 195, 'q': 7}, 'n': {'n': 8803, 'b': 43, 'h': 129, 'm': 131}, 'a': {'a': 9501, 's': 190, 'w': 62, 'q': 8, 'z': 4}, 'c': {'c': 4347, 'd': 76, 'f': 64, 'v': 34, 'x': 9}, 'v': {'c': 108, 'v': 1758, 'g': 49, 'b': 68, 'f': 68}, 'u': {'u': 3538, 'i': 273, 'y': 72, 'j': 3}, 'l': {'l': 5787, 'o': 311, 'p': 158, 'k': 10}, 's': {'s': 8809, 'd': 99, 'a': 353, 'z': 7, 'w': 53, 'x': 7}, 'f': {'f': 3015, 'r': 211, 'c': 118, 't': 318, 'v': 50, 'd': 113, 'g': 54}, 'm': {'m': 3390, 'n': 269, 'k': 14, 'j': 4}, 'q': {'a': 348, 'q': 135, 'w': 51}, 'p': {'l': 23

In [102]:
tmp = [list(x.keys()) for x in distribution_per_char.values()]

correct_c = [item for sublist in tmp for item in sublist]

typed_c_dict = Dict(distribution_per_char.keys())
correct_c_dict = Dict(correct_c)

## HMM ORDER 1

In [103]:
class HMM:
    def __init__(self, y_dict, x_dict):
        if not isinstance(y_dict, Dict) or not isinstance(x_dict, Dict):
            raise RuntimeError("Arguments must be of type Dict")

        self.y_dict = y_dict
        self.x_dict = x_dict

        n_y = len(y_dict)
        n_x = len(x_dict)
        self.init_prob = np.zeros((n_y,), float) 
        self.transition_prob = np.zeros((n_y, n_y), float) 
        self.observation_prob = np.zeros((n_y, n_x), float) 

In [104]:
print(correct_c_dict.char_to_id('e'))

11


In [105]:
hmm1 = HMM(typed_c_dict, correct_c_dict)

###init prob###
#computing the frequency of each tag to be the first tag of the sequence
for sent in train_data:
    tag = sent[0][1]
    id_tag = correct_c_dict.char_to_id(tag)
    hmm1.init_prob[id_tag]+=1
#smooting
hmm1.init_prob+=1
hmm1.init_prob/=(correct_c_dict.__len__()+len(train_data))
print(hmm1.init_prob)

###transition prob###
#computing p(yi|y(i-1)) 
d_tag = defaultdict(int)
for sent in train_data:
    for i in range(1,len(sent)):
        cur_char = sent[i][1]
        pred_char = sent[i-1][1]
        hmm1.transition_prob[correct_c_dict.char_to_id(pred_char)][correct_c_dict.char_to_id(cur_char)] += 1
        d_tag[correct_c_dict.char_to_id(pred_char)] +=1
        
#smoothing
hmm1.transition_prob+=1
for id_tag in d_tag:
    hmm1.transition_prob[id_tag,:]/=(d_tag[id_tag]+len(correct_c_dict))   
    
###observation prob###
d_tag = defaultdict(int)
for sent in train_data:
    for i in range(len(sent)):
        true_char = sent[i][1]
        typed_char = sent[i][0]
        true_char_tag = correct_c_dict.char_to_id(true_char)
        typed_char_tag = correct_c_dict.char_to_id(typed_char)
        hmm1.observation_prob[true_char_tag][typed_char_tag]+=1
        d_tag[true_char_tag]+=1
#smoothing
hmm1.observation_prob+=1
for id_tag in d_tag:
    hmm1.observation_prob[id_tag,:]/=(d_tag[id_tag]+len(correct_c_dict))

[4.91696180e-02 2.60633360e-02 1.68827150e-02 3.60691813e-02
 4.71065571e-03 2.57882612e-03 1.73950418e-01 9.93707664e-03
 2.60977203e-02 3.56221848e-02 1.89113915e-03 3.02926108e-02
 2.93642334e-02 5.44304233e-02 8.09063714e-02 7.88089262e-02
 2.75074786e-03 5.28143589e-02 2.32438194e-02 7.47171887e-02
 1.20345219e-03 4.02984561e-02 1.06385173e-01 3.43843482e-05
 4.16394457e-02 1.37537393e-04]


In [106]:
print(sum(hmm1.init_prob))
print(hmm1.transition_prob.sum(1))
print(hmm1.observation_prob.sum(1))

1.0000000000000002
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]


In [107]:
def viterbi(hmm, chars):
    """
    Input:
    - hmm: an HMM object
    - words: a list of words (ie a sentence)
    Return:
    - a list of POS tags
    """

    #DEFINING THE CHART AND BACKPOINTER TABLES
    chart = np.zeros((len(hmm.y_dict), len(chars)), float)
    backpointer = np.zeros((len(hmm.y_dict), len(chars)), float)
    
    #FILLING THE FIRST LINE OF THE CHART TABLE 
    for i in range(len(hmm.y_dict)):
        id_c0 = correct_c_dict.char_to_id(chars[0])
        chart[i,0] = hmm.init_prob[i]*hmm.observation_prob[i,id_c0]
        
    #FILLING BACKPOINTER TABLE AND THE REST OF THE CHART TABLE
    #for each char
    for i in range(1,len(chars)):
        id_c = correct_c_dict.char_to_id(chars[i])
        #for each possible char 
        for j in range(len(hmm.y_dict)):
            b_score = -100.0
            #for each possible (char, char') we want the maximum of the equation below
            for k in range(len(hmm.y_dict)):
                score = hmm.transition_prob[k,j]*hmm.observation_prob[j,id_c]*chart[k,i-1]
                #if the score is superior, we update because it wasn't the maximun
                if(score>b_score):
                    chart[j,i] = score
                    b_score = score
                    backpointer[j,i] = k
    
    #FILLING THE TABLE CONTAINING THE ID OF THE GOOD TAGS
    y = np.zeros(len(chars))
    y[len(chars)-1] = np.argmax(chart[:,len(chars)-1], axis=0)

    for j in range(1,len(chars))[::-1]:
        #print(j)
        y[j-1] = backpointer[int(y[j]),j]
    #MAPPING EACH ID TAG TO THE TAG
    pred = [correct_c_dict.id_to_char(int(i)) for i in (y)]

    return pred

In [108]:
# Evaluate the HMM using the viterbi
n_chars = 0
n_correct_chars = 0
n_correct_chars2 = 0

n_words = 0
n_correct_words = 0
n_correct_words2 = 0
for w in test_data:
    
    typed_chars = [c for c,_ in w]
    correct_chars = [c for _,c in w]
    pred = viterbi(hmm1, typed_chars)
    
    n_chars += len(typed_chars)
    
    correct_char_hmm = sum(1 for c in range(len(typed_chars))  if correct_chars[c] == pred[c])
    correct_char_nothing =  sum(1 for c in range(len(typed_chars))  if correct_chars[c] == typed_chars[c])
    
    n_correct_chars += correct_char_hmm
    n_correct_chars2 += correct_char_nothing

    n_words +=1
    if  len(typed_chars) == correct_char_hmm:
        n_correct_words += 1 
    if  len(typed_chars) == correct_char_nothing:
        n_correct_words2 += 1
    
print("Char Tagging accuracy for HMM Order 1: %.2f" % (100 * n_correct_chars / n_chars))
print("Word Tagging accuracy for HMM Order 1: %.2f" % (100 * n_correct_words / n_words))

print("Char Tagging accuracy before HMM Order 1: %.2f" % (100 * n_correct_chars2 / n_chars))
print("Word Tagging accuracy before HMM Order 1: %.2f" % (100 * n_correct_words2 / n_words))

Char Tagging accuracy for HMM Order 1: 93.21
Word Tagging accuracy for HMM Order 1: 75.08
Char Tagging accuracy before HMM Order 1: 89.82
Word Tagging accuracy before HMM Order 1: 62.89


In [109]:
# Evaluate the HMM using the viterbi
error_created = 0
error_corrected = 0
error_tot = 0
for w in test_data:
    
    typed_chars = [c for c,_ in w]
    correct_chars = [c for _,c in w]
    pred = viterbi(hmm1, typed_chars)
    
    for i in range (len(pred)):
        if ((typed_chars[i] == correct_chars[i]) & (pred[i] != correct_chars[i])):
            error_created += 1
        if (typed_chars[i] != correct_chars[i]) :
            error_tot += 1
            if (pred[i] == correct_chars[i]):
                error_corrected +=1
        
    
    
print("Error created by HMM Order 1: %.2f" % error_created)
print("Error corrected by HMM Order 1: %.2f percent" % (100 * error_corrected/error_tot) )


Error created by HMM Order 1: 62.00
Error corrected by HMM Order 1: 41.61 percent


## HMM ORDER 2

In [24]:
class HMM_2:
    def __init__(self, y_dict, x_dict):
        if not isinstance(y_dict, Dict) or not isinstance(x_dict, Dict):
            raise RuntimeError("Arguments must be of type Dict")

        self.y_dict = y_dict
        self.x_dict = x_dict

        n_y = len(y_dict)
        n_x = len(x_dict)
        self.init_prob = np.zeros((n_y,), float) 
        self.transition_prob1 = np.zeros((n_y,n_y), float)
        self.transition_prob = np.zeros((n_y, n_y,n_y), float) 
        self.observation_prob = np.zeros((n_y, n_x), float) 

In [25]:
hmm = HMM_2(typed_c_dict, correct_c_dict)

###init prob###
#computing the frequency of each tag to be the first tag of the sequence
for sent in train_data:
    tag = sent[0][1]
    id_tag = correct_c_dict.char_to_id(tag)
    hmm.init_prob[id_tag]+=1
#smooting
hmm.init_prob+=1
hmm.init_prob/=(correct_c_dict.__len__()+len(train_data))



###transition prob 1###
#computing p(yi|y(i-1)) 
d_tag = defaultdict(int)
for sent in train_data:
    for i in range(1,len(sent)):
        cur_char = sent[i][1]
        pred_char = sent[i-1][1]
        hmm.transition_prob1[correct_c_dict.char_to_id(pred_char)][correct_c_dict.char_to_id(cur_char)] += 1
        d_tag[correct_c_dict.char_to_id(pred_char)] +=1
        
#smoothing
hmm.transition_prob1+=1
for id_tag in d_tag:
    hmm.transition_prob1[id_tag,:]/=(d_tag[id_tag]+len(correct_c_dict))   
    
    
    

###transition prob###
#computing p(yi|y(i-1),y(i-2)) 
d_tag1 = defaultdict(int)
for sent in train_data:
    for i in range(2,len(sent)):
        cur_char = sent[i][1]
        pred_char = sent[i-1][1]
        pred2_char = sent[i-2][1]
        hmm.transition_prob[correct_c_dict.char_to_id(pred2_char)][correct_c_dict.char_to_id(pred_char)][correct_c_dict.char_to_id(cur_char)] += 1
        d_tag1[(correct_c_dict.char_to_id(pred2_char),correct_c_dict.char_to_id(pred_char))] +=1
 

#smoothing
hmm.transition_prob+=1
for i in range(26):
    for j in range(26):
        if (d_tag1[i,j]==0):
            hmm.transition_prob[i,j,:]=0
        else:
            hmm.transition_prob[i, j,:]/=(d_tag1[i, j]+len(correct_c_dict))   
    
###observation prob###
d_tag = defaultdict(int)
for sent in train_data:
    for i in range(len(sent)):
        true_char = sent[i][1]
        typed_char = sent[i][0]
        true_char_tag = correct_c_dict.char_to_id(true_char)
        typed_char_tag = correct_c_dict.char_to_id(typed_char)
        hmm.observation_prob[true_char_tag][typed_char_tag]+=1
        d_tag[true_char_tag]+=1
#smoothing
hmm.observation_prob+=1
for id_tag in d_tag:
    hmm.observation_prob[id_tag,:]/=(d_tag[id_tag]+len(correct_c_dict))

In [26]:
print((hmm.transition_prob[3,0,:]).sum())

0.9999999999999999


In [27]:
hmm.transition_prob.shape

(26, 26, 26)

In [28]:
def viterbi2(hmm, chars, viterbi):
    """
    Input:
    - hmm: an HMM object
    - words: a list of words (ie a sentence)
    Return:
    - a list of POS tags
    """
    
    #if two chars or less it is useless to do order 2 HMM so we call order 1 hmm as before
    if len(chars)<2:
        return(viterbi(hmm1, chars))
       
    ####if more than two chars##### 
    
    #INIT OF VITERBI & BACK POINTER TABLES
    #dim 26 * 26 * len(chars) ie the two hidden states and the typed char
    chart = np.zeros((len(hmm.y_dict),len(hmm.y_dict),len(chars)))
    tmp = np.zeros((len(hmm.y_dict),len(chars))) 
    back_ptr = np.zeros((len(hmm.y_dict),len(hmm.y_dict),len(chars)))



    
    ##chart[y, 0] = p(y)p(y |x0)
    #FILLING THE FIRST LINE OF THE CHART TABLE CORRESPONDING TO THE FIRST CHAR
    for i in range(len(hmm.y_dict)):
        id_c0 = correct_c_dict.char_to_id(chars[0])
        tmp[i,0] = hmm.init_prob[i]*hmm.observation_prob[i,id_c0]
        

    #FILLING THE FIRST TABLE OF THE CHART TABLE CORRESPONDING TO THE TWO LAST CHARS
    id_c1 = correct_c_dict.char_to_id(chars[1])
    for s in range (0,len(hmm.y_dict)):
        for v in range (0,len(hmm.y_dict)): 
            chart[s,v,1] = tmp[s,0] * hmm.transition_prob1[s,v] * hmm.observation_prob[id_c1,v]



   
    
    #for each char
    for i in range(2,len(chars)):
        id_c = correct_c_dict.char_to_id(chars[i])
        #for each possible char 
        for j in range (0,len(hmm.y_dict)): 
            #for each possible char before
            for k in range (0,len(hmm.y_dict)): 
                #finding the argmax and max to fill chart and backptr
                tmp_ = np.array(chart[:,j,i-1] * hmm.transition_prob[:,j,k]*hmm.observation_prob[k,id_c])
                chart[j,k,i] = (np.max(tmp_))
                back_ptr[j,k,i]= np.where(tmp_ == np.max(tmp_))[0]

                
    #taking the table corresponding to the two last chars : line is the ante last char (y(n-1))
    #and column is the last char (y(n))
    last_chars = chart[:,:,len(chars)-1]
    #taking the argmax : this argmax describes the two last chars
    
    y = np.zeros(len(chars))
    
    b_score = -1
    for i in range(26):
        for j in range(26):
            #searching for the argmax and the corresponding characters ie the position
            score = last_chars[i][j]
            if(score>b_score):
                b_score=score
                y[len(chars)-1] = j
                y[len(chars)-2] = i
                


    #y^_j =bckptr[y^_j+1,y^_j+2, j + 2]
    for j in range(0,len(chars)-2)[::-1]:
        y[j] = back_ptr[int(y[j+1]),int(y[j+2]),j+2]

    pred = [correct_c_dict.id_to_char(int(i)) for i in (y)]
    return pred

In [29]:
viterbi2(hmm, ['e','m','g','k','i','s','h'] ,viterbi)

['e', 'n', 'g', 'l', 'i', 's', 'h']

In [30]:
# Evaluate the HMM using the viterbi
n_chars = 0
n_correct_chars = 0
n_correct_chars2 = 0

n_words = 0
n_correct_words = 0
n_correct_words2 = 0
i=0
for w in test_data:
    i+=1
    if(i%100 == 0):
        print(i)
    typed_chars = [c for c,_ in w]
    correct_chars = [c for _,c in w]
    pred = viterbi2(hmm, typed_chars , viterbi)
    
    n_chars += len(typed_chars)
    
    correct_char_hmm = sum(1 for c in range(len(typed_chars))  if correct_chars[c] == pred[c])
    correct_char_nothing =  sum(1 for c in range(len(typed_chars))  if correct_chars[c] == typed_chars[c])
    
    n_correct_chars += correct_char_hmm
    n_correct_chars2 += correct_char_nothing

    n_words +=1
    if  len(typed_chars) == correct_char_hmm:
        n_correct_words += 1 
    if  len(typed_chars) == correct_char_nothing:
        n_correct_words2 += 1
    
print("Char Tagging accuracy for HMM Order 1: %.2f" % (100 * n_correct_chars / n_chars))
print("Word Tagging accuracy for HMM Order 1: %.2f" % (100 * n_correct_words / n_words))

print("Char Tagging accuracy before HMM Order 1: %.2f" % (100 * n_correct_chars2 / n_chars))
print("Word Tagging accuracy before HMM Order 1: %.2f" % (100 * n_correct_words2 / n_words))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
Char Tagging accuracy for HMM Order 1: 95.07
Word Tagging accuracy for HMM Order 1: 81.48
Char Tagging accuracy before HMM Order 1: 89.82
Word Tagging accuracy before HMM Order 1: 62.89


In [41]:
# Evaluate the HMM using the viterbi
error_created = 0
error_corrected = 0
error_tot = 0
for w in test_data:
    
    typed_chars = [c for c,_ in w]
    correct_chars = [c for _,c in w]
    pred = viterbi2(hmm, typed_chars, viterbi)
    
    for i in range (len(pred)):
        if ((typed_chars[i] == correct_chars[i]) & (pred[i] != correct_chars[i])):
            error_created += 1
        if (typed_chars[i] != correct_chars[i]) :
            error_tot += 1
            if (pred[i] == correct_chars[i]):
                error_corrected +=1
        
    
    
print("Error created by HMM Order 2: %.2f" % error_created)
print("Error corrected by HMM Order 2: %.2f percent" % (100 * error_corrected/error_tot) )


Error created by HMM Order 1: 80.00
Error corrected by HMM Order 1: 62.28 percent


# TYPOS CORRECTION WITH 20 % ERROR RATE

## HMM ORDER 1

In [48]:
distribution_per_char = {}
distribution_per_char_correct = {}
for w in train_data_20:
    for typed_c, correct_c in w:
        if typed_c in distribution_per_char : 
            if correct_c in distribution_per_char[typed_c]:
                distribution_per_char[typed_c][correct_c]+=1
            else:
                distribution_per_char[typed_c][correct_c] = 1
        else:
            distribution_per_char[typed_c]={}
            distribution_per_char[typed_c][correct_c] = 1

In [49]:
print(distribution_per_char)

{'b': {'b': 1576, 'h': 274, 'n': 478, 'v': 75, 'g': 107}, 'y': {'y': 2240, 't': 611, 'u': 176, 'h': 238}, 'j': {'h': 244, 'm': 232, 'u': 164, 'j': 78, 'i': 490, 'n': 462, 'k': 23}, 'w': {'e': 1130, 'w': 1677, 's': 349, 'q': 16}, 'i': {'i': 8174, 'o': 542, 'k': 21, 'u': 193}, 'r': {'r': 6135, 'e': 1096, 't': 660, 'f': 90}, 'o': {'o': 8913, 'i': 537, 'l': 393, 'p': 302}, 'q': {'w': 107, 'a': 675, 'q': 114}, 'n': {'n': 7291, 'h': 283, 'm': 220, 'j': 3, 'b': 104}, 'a': {'a': 7873, 's': 387, 'w': 96, 'q': 13, 'z': 5}, 'c': {'c': 3600, 'f': 104, 'v': 82, 'x': 10, 'd': 175}, 'f': {'c': 239, 't': 642, 'f': 2512, 'r': 380, 'v': 96, 'd': 144, 'g': 106}, 'u': {'u': 2992, 'i': 472, 'y': 117, 'j': 10}, 'v': {'v': 1431, 'c': 243, 'f': 89, 'b': 92, 'g': 92}, 'l': {'l': 4801, 'o': 580, 'p': 308, 'k': 19}, 'd': {'e': 1122, 'd': 3437, 'r': 406, 's': 365, 'c': 203, 'x': 17, 'f': 96}, 's': {'s': 7243, 'a': 681, 'd': 173, 'z': 11, 'x': 16, 'w': 109}, 't': {'t': 10379, 'f': 101, 'y': 132, 'r': 421, 'g': 105

In [50]:
tmp = [list(x.keys()) for x in distribution_per_char.values()]

correct_c = [item for sublist in tmp for item in sublist]

typed_c_dict = Dict(distribution_per_char.keys())
correct_c_dict = Dict(correct_c)

In [52]:
class HMM:
    def __init__(self, y_dict, x_dict):
        if not isinstance(y_dict, Dict) or not isinstance(x_dict, Dict):
            raise RuntimeError("Arguments must be of type Dict")

        self.y_dict = y_dict
        self.x_dict = x_dict

        n_y = len(y_dict)
        n_x = len(x_dict)
        self.init_prob = np.zeros((n_y,), float) 
        self.transition_prob = np.zeros((n_y, n_y), float) 
        self.observation_prob = np.zeros((n_y, n_x), float) 

In [54]:
hmm1 = HMM(typed_c_dict, correct_c_dict)

###init prob###
#computing the frequency of each tag to be the first tag of the sequence
for sent in train_data_20:
    tag = sent[0][1]
    id_tag = correct_c_dict.char_to_id(tag)
    hmm1.init_prob[id_tag]+=1
#smooting
hmm1.init_prob+=1
hmm1.init_prob/=(correct_c_dict.__len__()+len(train_data_20))
print(hmm1.init_prob)

###transition prob###
#computing p(yi|y(i-1)) 
d_tag = defaultdict(int)
for sent in train_data_20:
    for i in range(1,len(sent)):
        cur_char = sent[i][1]
        pred_char = sent[i-1][1]
        hmm1.transition_prob[correct_c_dict.char_to_id(pred_char)][correct_c_dict.char_to_id(cur_char)] += 1
        d_tag[correct_c_dict.char_to_id(pred_char)] +=1
        
#smoothing
hmm1.transition_prob+=1
for id_tag in d_tag:
    hmm1.transition_prob[id_tag,:]/=(d_tag[id_tag]+len(correct_c_dict))   
    
###observation prob###
d_tag = defaultdict(int)
for sent in train_data_20:
    for i in range(len(sent)):
        true_char = sent[i][1]
        typed_char = sent[i][0]
        true_char_tag = correct_c_dict.char_to_id(true_char)
        typed_char_tag = correct_c_dict.char_to_id(typed_char)
        hmm1.observation_prob[true_char_tag][typed_char_tag]+=1
        d_tag[true_char_tag]+=1
#smoothing
hmm1.observation_prob+=1
for id_tag in d_tag:
    hmm1.observation_prob[id_tag,:]/=(d_tag[id_tag]+len(correct_c_dict))

[4.95406101e-02 3.54649026e-02 2.63873576e-02 4.81440647e-03
 1.68687982e-02 2.75633958e-03 1.74788681e-01 1.02168320e-02
 4.03528115e-02 2.02131569e-03 8.06321205e-02 2.79309078e-03
 3.08710033e-02 5.38037486e-02 7.53031974e-02 1.21278942e-03
 7.82432929e-02 2.62036016e-02 3.59426681e-02 2.22344726e-02
 5.36567438e-02 1.05181918e-01 3.67511944e-05 4.15288497e-02
 1.47004778e-04 2.89966924e-02]


In [55]:
for i in range (hmm1.observation_prob.shape[0]):
    print(hmm1.observation_prob[i][i])

0.7992904206791688
0.7911633820724729
0.7942489924844788
0.7876787678767877
0.8007766990291262
0.8012155881301395
0.8009877305347635
0.8004814121422841
0.8002241524236481
0.632
0.7996674166096058
0.8034188034188035
0.8007090103397341
0.7967711301044634
0.7962189492196087
0.6804733727810651
0.7936960199447957
0.7911294481691593
0.8028753993610224
0.801669449081803
0.7930922124470512
0.7948717948717948
0.6573426573426573
0.7936962750716332
0.7202797202797203
0.8012118387322302


In [56]:
print(sum(hmm1.init_prob))
print(hmm1.transition_prob.sum(1))
print(hmm1.observation_prob.sum(1))

1.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]


In [57]:
print((hmm1.observation_prob[21]))

[1.00948920e-04 1.00948920e-04 1.00948920e-04 1.00948920e-04
 1.00948920e-04 1.00948920e-04 1.00948920e-04 1.00948920e-04
 1.00948920e-04 1.00948920e-04 1.00948920e-04 1.00948920e-04
 1.00948920e-04 1.00948920e-04 6.88471633e-02 6.82414698e-02
 1.00948920e-04 1.00948920e-04 1.00948920e-04 1.00948920e-04
 1.00948920e-04 7.94871795e-01 6.58186957e-02 1.00948920e-04
 1.00948920e-04 1.00948920e-04]


In [58]:
def viterbi(hmm, chars):
    """
    Input:
    - hmm: an HMM object
    - words: a list of words (ie a sentence)
    Return:
    - a list of POS tags
    """

    #DEFINING THE CHART AND BACKPOINTER TABLES
    chart = np.zeros((len(hmm.y_dict), len(chars)), float)
    backpointer = np.zeros((len(hmm.y_dict), len(chars)), float)
    
    #FILLING THE FIRST LINE OF THE CHART TABLE 
    for i in range(len(hmm.y_dict)):
        id_c0 = correct_c_dict.char_to_id(chars[0])
        chart[i,0] = hmm.init_prob[i]*hmm.observation_prob[i,id_c0]
        
    #FILLING BACKPOINTER TABLE AND THE REST OF THE CHART TABLE
    #for each char
    for i in range(1,len(chars)):
        id_c = correct_c_dict.char_to_id(chars[i])
        #for each possible char 
        for j in range(len(hmm.y_dict)):
            b_score = -100.0
            #for each possible (char, char') we want the maximum of the equation below
            for k in range(len(hmm.y_dict)):
                score = hmm.transition_prob[k,j]*hmm.observation_prob[j,id_c]*chart[k,i-1]
                #if the score is superior, we update because it wasn't the maximun
                if(score>b_score):
                    chart[j,i] = score
                    b_score = score
                    backpointer[j,i] = k
    
    #FILLING THE TABLE CONTAINING THE ID OF THE GOOD TAGS
    y = np.zeros(len(chars))
    y[len(chars)-1] = np.argmax(chart[:,len(chars)-1], axis=0)

    for j in range(1,len(chars))[::-1]:
        #print(j)
        y[j-1] = backpointer[int(y[j]),j]
    #MAPPING EACH ID TAG TO THE TAG
    pred = [correct_c_dict.id_to_char(int(i)) for i in (y)]

    return pred

In [60]:
# Evaluate the HMM using the viterbi
n_chars = 0
n_correct_chars = 0
n_correct_chars2 = 0

n_words = 0
n_correct_words = 0
n_correct_words2 = 0
for w in test_data_20:
    
    typed_chars = [c for c,_ in w]
    correct_chars = [c for _,c in w]
    pred = viterbi(hmm1, typed_chars)
    
    n_chars += len(typed_chars)
    
    correct_char_hmm = sum(1 for c in range(len(typed_chars))  if correct_chars[c] == pred[c])
    correct_char_nothing =  sum(1 for c in range(len(typed_chars))  if correct_chars[c] == typed_chars[c])
    
    n_correct_chars += correct_char_hmm
    n_correct_chars2 += correct_char_nothing

    n_words +=1
    if  len(typed_chars) == correct_char_hmm:
        n_correct_words += 1 
    if  len(typed_chars) == correct_char_nothing:
        n_correct_words2 += 1
    
print("Char Tagging accuracy for HMM Order 1: %.2f" % (100 * n_correct_chars / n_chars))
print("Word Tagging accuracy for HMM Order 1: %.2f" % (100 * n_correct_words / n_words))

print("Char Tagging accuracy before HMM Order 1: %.2f" % (100 * n_correct_chars2 / n_chars))
print("Word Tagging accuracy before HMM Order 1: %.2f" % (100 * n_correct_words2 / n_words))

Char Tagging accuracy for HMM Order 1: 86.87
Word Tagging accuracy for HMM Order 1: 58.00
Char Tagging accuracy before HMM Order 1: 80.59
Word Tagging accuracy before HMM Order 1: 40.66


In [61]:
# Evaluate the HMM using the viterbi
error_created = 0
error_corrected = 0
error_tot = 0
for w in test_data_20:
    
    typed_chars = [c for c,_ in w]
    correct_chars = [c for _,c in w]
    pred = viterbi(hmm1, typed_chars)
    
    for i in range (len(pred)):
        if ((typed_chars[i] == correct_chars[i]) & (pred[i] != correct_chars[i])):
            error_created += 1
        if (typed_chars[i] != correct_chars[i]) :
            error_tot += 1
            if (pred[i] == correct_chars[i]):
                error_corrected +=1
        
    
    
print("Error created by HMM Order 1: %.2f" % error_created)
print("Error corrected by HMM Order 1: %.2f percent" % (100 * error_corrected/error_tot) )


Error created by HMM Order 1: 314.00
Error corrected by HMM Order 1: 42.02 percent


## HMM ORDER 2

In [62]:
class HMM_2:
    def __init__(self, y_dict, x_dict):
        if not isinstance(y_dict, Dict) or not isinstance(x_dict, Dict):
            raise RuntimeError("Arguments must be of type Dict")

        self.y_dict = y_dict
        self.x_dict = x_dict

        n_y = len(y_dict)
        n_x = len(x_dict)
        self.init_prob = np.zeros((n_y,), float) 
        self.transition_prob1 = np.zeros((n_y,n_y), float)
        self.transition_prob = np.zeros((n_y, n_y,n_y), float) 
        self.observation_prob = np.zeros((n_y, n_x), float) 

In [68]:
hmm = HMM_2(typed_c_dict, correct_c_dict)

###init prob###
#computing the frequency of each tag to be the first tag of the sequence
for sent in train_data_20:
    tag = sent[0][1]
    id_tag = correct_c_dict.char_to_id(tag)
    hmm.init_prob[id_tag]+=1
#smooting
hmm.init_prob+=1
hmm.init_prob/=(correct_c_dict.__len__()+len(train_data_20))



###transition prob 1###
#computing p(yi|y(i-1)) 
d_tag = defaultdict(int)
for sent in train_data_20:
    for i in range(1,len(sent)):
        cur_char = sent[i][1]
        pred_char = sent[i-1][1]
        hmm.transition_prob1[correct_c_dict.char_to_id(pred_char)][correct_c_dict.char_to_id(cur_char)] += 1
        d_tag[correct_c_dict.char_to_id(pred_char)] +=1
        
#smoothing
hmm.transition_prob1+=1
for id_tag in d_tag:
    hmm.transition_prob1[id_tag,:]/=(d_tag[id_tag]+len(correct_c_dict))   
    
    
    

###transition prob###
#computing p(yi|y(i-1),y(i-2)) 
d_tag1 = defaultdict(int)
for sent in train_data_20:
    for i in range(2,len(sent)):
        cur_char = sent[i][1]
        pred_char = sent[i-1][1]
        pred2_char = sent[i-2][1]
        hmm.transition_prob[correct_c_dict.char_to_id(pred2_char)][correct_c_dict.char_to_id(pred_char)][correct_c_dict.char_to_id(cur_char)] += 1
        d_tag1[(correct_c_dict.char_to_id(pred2_char),correct_c_dict.char_to_id(pred_char))] +=1
 

#smoothing
hmm.transition_prob+=1
for i in range(26):
    for j in range(26):
        if (d_tag1[i,j]==0):
            hmm.transition_prob[i,j,:]=0
        else:
            hmm.transition_prob[i, j,:]/=(d_tag1[i, j]+len(correct_c_dict))   
    
###observation prob###
d_tag = defaultdict(int)
for sent in train_data_20:
    for i in range(len(sent)):
        true_char = sent[i][1]
        typed_char = sent[i][0]
        true_char_tag = correct_c_dict.char_to_id(true_char)
        typed_char_tag = correct_c_dict.char_to_id(typed_char)
        hmm.observation_prob[true_char_tag][typed_char_tag]+=1
        d_tag[true_char_tag]+=1
#smoothing
hmm.observation_prob+=1
for id_tag in d_tag:
    hmm.observation_prob[id_tag,:]/=(d_tag[id_tag]+len(correct_c_dict))

In [69]:
hmm.transition_prob.shape

(26, 26, 26)

In [70]:
def viterbi2(hmm, chars, viterbi):
    """
    Input:
    - hmm: an HMM object
    - words: a list of words (ie a sentence)
    Return:
    - a list of POS tags
    """
    
    #if two chars or less it is useless to do order 2 HMM so we call order 1 hmm as before
    if len(chars)<2:
        return(viterbi(hmm1, chars))
       
    ####if more than two chars##### 
    
    #INIT OF VITERBI & BACK POINTER TABLES
    #dim 26 * 26 * len(chars) ie the two hidden states and the typed char
    chart = np.zeros((len(hmm.y_dict),len(hmm.y_dict),len(chars)))
    tmp = np.zeros((len(hmm.y_dict),len(chars))) 
    back_ptr = np.zeros((len(hmm.y_dict),len(hmm.y_dict),len(chars)))



    
    ##chart[y, 0] = p(y)p(y |x0)
    #FILLING THE FIRST LINE OF THE CHART TABLE CORRESPONDING TO THE FIRST CHAR
    for i in range(len(hmm.y_dict)):
        id_c0 = correct_c_dict.char_to_id(chars[0])
        tmp[i,0] = hmm.init_prob[i]*hmm.observation_prob[i,id_c0]
        

    #FILLING THE FIRST TABLE OF THE CHART TABLE CORRESPONDING TO THE TWO LAST CHARS
    id_c1 = correct_c_dict.char_to_id(chars[1])
    for s in range (0,len(hmm.y_dict)):
        for v in range (0,len(hmm.y_dict)): 
            chart[s,v,1] = tmp[s,0] * hmm.transition_prob1[s,v] * hmm.observation_prob[id_c1,v]



   
    
    #for each char
    for i in range(2,len(chars)):
        id_c = correct_c_dict.char_to_id(chars[i])
        #for each possible char 
        for j in range (0,len(hmm.y_dict)): 
            #for each possible char before
            for k in range (0,len(hmm.y_dict)): 
                #finding the argmax and max to fill chart and backptr
                tmp_ = np.array(chart[:,j,i-1] * hmm.transition_prob[:,j,k]*hmm.observation_prob[k,id_c])
                chart[j,k,i] = (np.max(tmp_))
                back_ptr[j,k,i]= np.where(tmp_ == np.max(tmp_))[0]

                
    #taking the table corresponding to the two last chars : line is the ante last char (y(n-1))
    #and column is the last char (y(n))
    last_chars = chart[:,:,len(chars)-1]
    #taking the argmax : this argmax describes the two last chars
    
    y = np.zeros(len(chars))
    
    b_score = -1
    for i in range(26):
        for j in range(26):
            #searching for the argmax and the corresponding characters ie the position
            score = last_chars[i][j]
            if(score>b_score):
                b_score=score
                y[len(chars)-1] = j
                y[len(chars)-2] = i
                


    #y^_j =bckptr[y^_j+1,y^_j+2, j + 2]
    for j in range(0,len(chars)-2)[::-1]:
        y[j] = back_ptr[int(y[j+1]),int(y[j+2]),j+2]

    pred = [correct_c_dict.id_to_char(int(i)) for i in (y)]
    return pred

In [71]:
viterbi2(hmm, ['e','m','g','k','i','s','h'] ,viterbi)

['e', 'n', 'g', 'l', 'i', 's', 'h']

In [74]:
# Evaluate the HMM using the viterbi
n_chars = 0
n_correct_chars = 0
n_correct_chars2 = 0

n_words = 0
n_correct_words = 0
n_correct_words2 = 0


for w in test_data_20:
    typed_chars = [c for c,_ in w]
    correct_chars = [c for _,c in w]
    pred = viterbi2(hmm, typed_chars , viterbi)
    
    n_chars += len(typed_chars)
    
    correct_char_hmm = sum(1 for c in range(len(typed_chars))  if correct_chars[c] == pred[c])
    correct_char_nothing =  sum(1 for c in range(len(typed_chars))  if correct_chars[c] == typed_chars[c])
    
    n_correct_chars += correct_char_hmm
    n_correct_chars2 += correct_char_nothing

    n_words +=1
    if  len(typed_chars) == correct_char_hmm:
        n_correct_words += 1 
    if  len(typed_chars) == correct_char_nothing:
        n_correct_words2 += 1
    
print("Char Tagging accuracy for HMM Order 1: %.2f" % (100 * n_correct_chars / n_chars))
print("Word Tagging accuracy for HMM Order 1: %.2f" % (100 * n_correct_words / n_words))

print("Char Tagging accuracy before HMM Order 1: %.2f" % (100 * n_correct_chars2 / n_chars))
print("Word Tagging accuracy before HMM Order 1: %.2f" % (100 * n_correct_words2 / n_words))

Char Tagging accuracy for HMM Order 1: 91.35
Word Tagging accuracy for HMM Order 1: 71.93
Char Tagging accuracy before HMM Order 1: 80.59
Word Tagging accuracy before HMM Order 1: 40.66


In [75]:
# Evaluate the HMM using the viterbi
error_created = 0
error_corrected = 0
error_tot = 0
for w in test_data_20:
    
    typed_chars = [c for c,_ in w]
    correct_chars = [c for _,c in w]
    pred = viterbi2(hmm, typed_chars, viterbi)
    
    for i in range (len(pred)):
        if ((typed_chars[i] == correct_chars[i]) & (pred[i] != correct_chars[i])):
            error_created += 1
        if (typed_chars[i] != correct_chars[i]) :
            error_tot += 1
            if (pred[i] == correct_chars[i]):
                error_corrected +=1
        
    
    
print("Error created by HMM Order 1: %.2f" % error_created)
print("Error corrected by HMM Order 1: %.2f percent" % (100 * error_corrected/error_tot) )


Error created by HMM Order 1: 313.00
Error corrected by HMM Order 1: 65.11 percent


# Add noisy characters

In [30]:
train_data [:10]

[[('b', 'b'), ('y', 'y')],
 [('t', 't'), ('h', 'h'), ('e', 'e'), ('i', 'i'), ('r', 'r')],
 [('o', 'o'), ('w', 'w'), ('n', 'n')],
 [('a', 'a'),
  ('c', 'c'),
  ('v', 'c'),
  ('o', 'o'),
  ('u', 'u'),
  ('n', 'n'),
  ('t', 't')],
 [('v', 'v'),
  ('i', 'i'),
  ('o', 'o'),
  ('l', 'l'),
  ('e', 'e'),
  ('n', 'n'),
  ('c', 'c'),
  ('e', 'e')],
 [('i', 'i'), ('s', 's')],
 [('f', 'f'), ('o', 'o'), ('r', 'r')],
 [('t', 't'), ('h', 'h'), ('e', 'e'), ('m', 'm')],
 [('q', 'a')],
 [('f', 'f'), ('o', 'o'), ('r', 'r'), ('m', 'm')]]

In [58]:
train_data_bis = train_data
test_data_bis = test_data
for i in range(100):
    idx_word = np.random.randint(len(train_data_bis))
    train_data_bis[idx_word].insert(np.random.randint(len(train_data_bis[idx_word])) , ['a' , 'not'])

for i in range(100):
    idx_word = np.random.randint(len(test_data_bis))
    test_data_bis[idx_word].insert(np.random.randint(len(test_data_bis[idx_word])) , ['a' , 'not'])

In [59]:
distribution_per_char = {}
distribution_per_char_correct = {}
for w in train_data_bis:
    for typed_c, correct_c in w:
        if typed_c in distribution_per_char : 
            if correct_c in distribution_per_char[typed_c]:
                distribution_per_char[typed_c][correct_c]+=1
            else:
                distribution_per_char[typed_c][correct_c] = 1
        else:
            distribution_per_char[typed_c]={}
            distribution_per_char[typed_c][correct_c] = 1

In [62]:
tmp = [list(x.keys()) for x in distribution_per_char.values()]

correct_c = [item for sublist in tmp for item in sublist]

typed_c_dict = Dict(distribution_per_char.keys())
correct_c_dict = Dict(correct_c)

In [64]:
print(len(typed_c_dict))
print(len(correct_c_dict))

26
27


In [76]:
hmm_bis = HMM(correct_c_dict , typed_c_dict)

###init prob###
#computing the frequency of each tag to be the first tag of the sequence
for sent in train_data:
    tag = sent[0][1]
    id_tag = correct_c_dict.char_to_id(tag)
    hmm_bis.init_prob[id_tag]+=1
#smooting
hmm_bis.init_prob+=1
hmm_bis.init_prob/=(correct_c_dict.__len__()+len(train_data))
print(hmm_bis.init_prob)

###transition prob###
#computing p(yi|y(i-1)) 
d_tag = defaultdict(int)
for sent in train_data:
    for i in range(1,len(sent)):
        cur_char = sent[i][1]
        pred_char = sent[i-1][1]
        hmm_bis.transition_prob[correct_c_dict.char_to_id(pred_char)][correct_c_dict.char_to_id(cur_char)] += 1
        d_tag[correct_c_dict.char_to_id(pred_char)] +=1
        
#smoothing
hmm_bis.transition_prob+=1
for id_tag in d_tag:
    hmm_bis.transition_prob[id_tag,:]/=(d_tag[id_tag]+len(correct_c_dict))   
    
###observation prob###
d_tag = defaultdict(int)
for sent in train_data:
    for i in range(len(sent)):
        true_char = sent[i][1]
        typed_char = sent[i][0]
        true_char_tag = correct_c_dict.char_to_id(true_char)
        typed_char_tag = typed_c_dict.char_to_id(typed_char)
        hmm_bis.observation_prob[true_char_tag][typed_char_tag]+=1
        d_tag[true_char_tag]+=1
#smoothing
hmm_bis.observation_prob+=1
for id_tag in d_tag:
    hmm_bis.observation_prob[id_tag,:]/=(d_tag[id_tag]+len(correct_c_dict))

27
[4.90303947e-02 2.59592903e-02 1.68477513e-02 3.59304085e-02
 4.71049374e-03 2.54435428e-03 1.73566222e-01 9.90235181e-03
 2.60968230e-02 3.55865768e-02 1.85669096e-03 3.02228029e-02
 2.93288406e-02 5.42910191e-02 8.05597579e-02 7.84967680e-02
 2.75065328e-03 5.27781598e-02 2.32430202e-02 7.45770871e-02
 1.20341081e-03 4.02283042e-02 1.06003301e-01 2.54435428e-03
 3.43831660e-05 4.15692477e-02 1.37532664e-04]


In [89]:
print(sum(hmm_bis.init_prob))
print(hmm_bis.transition_prob.sum(1))
print(hmm_bis.observation_prob.sum(1))

0.9999999999999998
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]
[0.99952313 0.99989801 0.99963807 0.99985097 0.99948823 0.99966799
 0.99992808 0.99974735 0.99987914 0.9997064  0.99259259 0.99994481
 0.99978109 0.99955674 0.99990912 0.9999164  0.99837925 0.99969174
 0.99984482 0.99989784 0.99435028 0.99973684 0.99990554 0.9969419
 0.99337748 0.99979317 0.99667774]


In [92]:
def viterbi(hmm, chars):
    """
    Input:
    - hmm: an HMM object
    - words: a list of words (ie a sentence)
    Return:
    - a list of POS tags
    """

    #DEFINING THE CHART AND BACKPOINTER TABLES
    chart = np.zeros((len(hmm.y_dict), len(chars)), float)
    backpointer = np.zeros((len(hmm.y_dict), len(chars)), float)
    
    #FILLING THE FIRST LINE OF THE CHART TABLE 
    for i in range(len(hmm.y_dict)):
        id_c0 = typed_c_dict.char_to_id(chars[0])
        chart[i,0] = hmm.init_prob[i]*hmm.observation_prob[i,id_c0]
        
    #FILLING BACKPOINTER TABLE AND THE REST OF THE CHART TABLE
    #for each char
    for i in range(1,len(chars)):
        id_c = typed_c_dict.char_to_id(chars[i])
        #for each possible char 
        for j in range(len(hmm.y_dict)):
            b_score = -100.0
            #for each possible (char, char') we want the maximum of the equation below
            for k in range(len(hmm.y_dict)):
                score = hmm.transition_prob[k,j]*hmm.observation_prob[j,id_c]*chart[k,i-1]
                #if the score is superior, we update because it wasn't the maximun
                if(score>b_score):
                    chart[j,i] = score
                    b_score = score
                    backpointer[j,i] = k
    
    #FILLING THE TABLE CONTAINING THE ID OF THE GOOD TAGS
    y = np.zeros(len(chars))
    y[len(chars)-1] = np.argmax(chart[:,len(chars)-1], axis=0)

    for j in range(1,len(chars))[::-1]:
        #print(j)
        y[j-1] = backpointer[int(y[j]),j]
    #MAPPING EACH ID TAG TO THE TAG
    pred = [correct_c_dict.id_to_char(int(i)) for i in (y)]

    return pred

In [93]:
# Evaluate the HMM using the viterbi
n_chars = 0
n_correct_chars = 0
n_correct_chars2 = 0

n_words = 0
n_correct_words = 0
n_correct_words2 = 0
for w in test_data_bis:
    
    typed_chars = [c for c,_ in w]
    correct_chars = [c for _,c in w]
    pred = viterbi(hmm1, typed_chars)
    n_chars += len(typed_chars)
    print(pred)
    print(typed_chars)
    correct_char_hmm = sum(1 for c in range(len(typed_chars))  if correct_chars[c] == pred[c])
    correct_char_nothing =  sum(1 for c in range(len(typed_chars))  if correct_chars[c] == typed_chars[c])
    
    n_correct_chars += correct_char_hmm
    n_correct_chars2 += correct_char_nothing

    n_words +=1
    if  len(typed_chars) == correct_char_hmm:
        n_correct_words += 1 
    if  len(typed_chars) == correct_char_nothing:
        n_correct_words2 += 1
    
print("Char Tagging accuracy for HMM Order 1: %.2f" % (100 * n_correct_chars / n_chars))
print("Word Tagging accuracy for HMM Order 1: %.2f" % (100 * n_correct_words / n_words))

print("Char Tagging accuracy before HMM Order 1: %.2f" % (100 * n_correct_chars2 / n_chars))
print("Word Tagging accuracy before HMM Order 1: %.2f" % (100 * n_correct_words2 / n_words))

['g', 'h', 'v']
['t', 'h', 'e']
['i', 'v', 'i', 'g', 'y', 'o', 'g']
['l', 'e', 'f', 't', 'i', 's', 't']
['t', 'o']
['i', 's']
['g', 'u', 'i']
['t', 'o', 'o']
['k', 'i', 't']
['f', 'a', 'r']
['a', 'u', 'r', 'v']
['g', 'o', 'n', 'e']
['o', 'u', 't']
['f', 'o', 'r']
['g', 'h', 'i', 'n', 'g']
['t', 'h', 'a', 'a', 't']
['h', 'y', 'o']
['h', 'i', 's']
['t', 'i', 'v', 'i', 'q', 'u', 's', 'o']
['r', 'e', 'e', 'l', 'j', 'h', 'g', 's']
['u', 'l']
['o', 'f']
['y', 'z', 'o', 'f', 't', 'y', 'z', 't', 'h', 'a', 'n']
['i', 'n', 'f', 'e', 'r', 'i', 'k', 'r', 'i', 'g', 'y']
['i', 't', 'v']
['a', 'r', 'e']
['o', 'n', 'i']
['s', 'a', 'l']
['y', 'z', 'a', 't', 'a', 'y', 'z', 'o', 'm']
['i', 'n', 'g', 'r', 'z', 'i', 'n', 'e', 'd']
['g', 'h', 'n', 'g']
['t', 'h', 'a', 't']
['b', 'v']
['h', 'e']
['e', 'n', 'f', 'f', 'u', 'g']
['c', 'a', 'n', 'n', 'o', 't']
['d', 'u', 'f', 'e', 'v', 'y', 'e', 'v']
['c', 'o', 'n', 'c', 'e', 'i', 'v', 'e']
['u', 'l']
['o', 'f']
['h', 'c', 'p', 'o', 'z', 'z', 'k']
['h', 'k', 'm'

['i', 'not', 'i', 'e', 'w', 'i', 'n', 'g', 'g', 'm']
['f', 'z', 'l', 'c', 'u', 'l', 'a', 't', 'e', 'd']
['g', 'u']
['t', 'o']
['i', 'b', 'j', 'v']
['a', 'b', 'a', 'e']
['u', 'l']
['o', 'f']
['b', 'v', 'f', 'f', 'o', 'u', 'g']
['b', 'e', 'n', 'e', 'f', 'i', 't']
['g', 'u']
['t', 'o']
['g', 'h', 'v']
['t', 'h', 'e']
['s', 't', 'u', 's', 'i', 'v']
['p', 'w', 'o', 'p', 'l', 'e']
['r', 'i', 'b', 'u', 'p']
['w', 'a', 'h', 'o', 'm']
['n', 'g', 'h', 'i', 'v']
['a', 't', 'h', 'a', 'e']
['i', 'v', 'i', 'g', 'y', 'o', 'g', 'i', 'o']
['l', 'e', 'f', 't', 'i', 's', 't', 'a', 's']
['z', 'i', 'n', 'y', 'p']
['x', 'l', 'a', 'i', 'm']
['g', 'u']
['t', 'o']
['i', 'b', 'j', 'v']
['a', 'b', 'a', 'e']
['g', 'r', 'n', 'y', 'z', 'a']
['t', 'r', 'y', 'i', 'n', 'g']
['g', 'u']
['t', 'o']
['n', 'g', 'h', 'i', 's']
['h', 'e', 'a', 'l', 'p']
['d', 'u', 't']
['v', 'o', 'r']
['v', 'c', 'i', 't', 's', 'i', 'v']
['e', 'x', 'a', 'n', 'p', 'l', 'e']
['t', 'i']
['i', 'f']
['u', 'r', 'v']
['o', 'n', 'e']
['b', 'j', 'u', 

['s', 't', 's', 'e', 'y', 'e', 'v']
['p', 'r', 'p', 'v', 'i', 'c', 'e']
['g', 'h', 'a', 'p', 'o', 'v', 'i', 'd', 'v', 'o']
['t', 'h', 'e', 'm', 's', 'e', 'l', 'v', 'e', 's']
['r', 'y', 'g', 'h']
['w', 'i', 't', 'h']
['i', 'f']
['a', 'n']
['v', 'c', 'e', 'w', 'o', 'v']
['e', 'x', 'c', 'u', 's', 'e']
['o', 'u', 't']
['f', 'o', 'r']
['q', 'u', 'z', 't', 'i', 'f', 'a']
['j', 'a', 'k', 'i', 'a', 'n', 'g']
['i']
['a']
['o', 'w', 'o', 'o']
['f', 'u', 's', 's']
['r', 'i', 'v']
['w', 'a', 'e']
['v', 'o', 's', 'u', 'l', 'i', 'o', 'u', 'a', 'v']
['e', 'm', 'p', 'h', 'q', 'a', 's', 'i', 'z', 'e']
['w', 'h', 'n', 'g']
['g', 'h', 'a', 't']
['n', 'n', 'v']
['y', 'h', 'e']
['o', 'u', 'r', 'v', 'a', 'u', 't', 'r', 'a']
['f', 'o', 'r', 'e', 'g', 'o', 'i', 'n', 'g']
['o', 'u', 'z', 'o']
['s', 'o', 'e', 's']
['f', 'u', 'g']
['n', 'o', 't']
['s', 'o', 'f', 'f', 'f', 'r', 'm']
['p', 'f', 'e', 't', 'e', 'n', 'd']
['g', 'u']
['t', 'o']
['b', 'v']
['b', 'e']
['i', 'f']
['a', 'n']
['l', 'e', 'e', 's', 't', 'i',

['p', 'u', 't', 'h', 'i', 'o', 'n']
['m', 'o', 'r', 'a', 'l', 'l', 'y']
['y', 'p', 's', 'u', 'l', 'v', 'o']
['i', 'm', 'p', 'o', 's', 'e', 's']
['i']
['a']
['o', 'r', 'a', 'v', 'e', 'v']
['s', 'w', 'g', 'e', 'r', 'e']
['b', 'e', 'n', 't', 'o', 'f', 'f']
['b', 'u', 'a', 'r', 'd', 'e', 'n']
['u', 'f']
['o', 'n']
['g', 'h', 'a', 'p']
['t', 'h', 'e', 'm']
['y', 'p']
['i', 'm']
['u', 't', 'm', 'a', 't']
['o', 'r', 'd', 'e', 'r']
['g', 'u']
['t', 'o']
['m', 'a', 'u', 'y', 'm']
['a', 'g', 'o', 'i', 'd']
['i', 'v', 'v', 'i', 't', 'r', 'a', 'l']
['f', 'e', 'e', 'l', 'i', 'n', 'g', 's']
['u', 'l']
['o', 'f']
['a', 's', 'y', 'i', 'g']
['g', 'u', 'i', 'l', 't']
['g', 'i', 'n', 'g', 'n']
['t', 'a', 'h', 'e', 'y']
['d', 'u', 'f', 'u', 'g', 'y', 'z', 'e', 'n', 'i', 'o', 'u', 'n']
['c', 'o', 'n', 'a', 't', 'i', 'n', 'u', 'a', 'l', 'l', 'a', 'y']
['h', 'i', 'a', 'r']
['h', 'a', 'g', 'w']
['b', 'j', 'u']
['t', 'a', 'o']
['m', 'v', 'c', 'z', 'h', 'e', 'm']
['d', 'e', 'x', 'e', 'i', 'v', 'd']
['g', 'h', '

KeyboardInterrupt: 

In [83]:
# Evaluate the HMM using the viterbi
error_created = 0
error_corrected = 0
error_tot = 0
for w in test_data_bis:
    
    typed_chars = [c for c,_ in w]
    correct_chars = [c for _,c in w]
    pred = viterbi(hmm_bis, typed_chars)
    
    for i in range (len(pred)):
        if ((typed_chars[i] == correct_chars[i]) & (pred[i] != correct_chars[i])):
            error_created += 1
        if (typed_chars[i] != correct_chars[i]) :
            error_tot += 1
            if (pred[i] == correct_chars[i]):
                error_corrected +=1
        
    
    
print("Error created by HMM Order 1: %.2f" % error_created)
print("Error corrected by HMM Order 1: %.2f percent" % (100 * error_corrected/error_tot) )


Error created by HMM Order 1: 196.00
Error corrected by HMM Order 1: 37.54 percent
